In [1]:
import yaml
import pandas as pd

from azure.core.credentials import AzureKeyCredential
from langchain.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
# Import config file
with open("config.yml", "r") as config_file:
    config = yaml.safe_load(config_file)

In [3]:
# Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = config['azure_openai_endpoint']
AZURE_OPENAI_API_KEY = config['azure_openai_key']
AZURE_OPENAI_API_VERSION = config['azure_openai_api_version']
AZURE_OPENAI_NAME_DEPLOYMENT = config['azure_openai_model_deployment']

# Azure AI search
AISEARCH_ENDPOINT = config['aisearch_endpoint']
AISEARCH_KEY = config['aisearch_credential']
AISEARCH_CREDENTIAL = AzureKeyCredential(config['aisearch_credential'])
AISEARCH_INDEX_NAME = config['aisearch_index_name']

# Azure embedding
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = config['azure_openai_embedding_deployment']
AZURE_OPENAI_EMBEDDING_MODEL_NAME = config['embedding_model_name']

In [4]:
# Will be used by langchain to generate response
chatmodel = AzureChatOpenAI(
            azure_endpoint= AZURE_OPENAI_ENDPOINT,
            openai_api_key=AZURE_OPENAI_API_KEY,
            openai_api_version=AZURE_OPENAI_API_VERSION,
            azure_deployment=AZURE_OPENAI_NAME_DEPLOYMENT,
            temperature=0,
            model_version="2024-07-18")

# Will be used by langchain to embed the question and do the search
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

#will be used for handling the vector store Azure AI Search 
vector_store = AzureSearch(
    azure_search_endpoint=AISEARCH_ENDPOINT,
    azure_search_key=AISEARCH_KEY,
    index_name=AISEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="my-semantic-config"
)

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [6]:
retriever = vector_store.as_retriever(
     k = 3, #top k matches
     #search_type = "hybrid" #"similarity" (default) vector similarity, "hybrid" Vectorial + Terms, "Semantic Hybrid" vectorial + semantic. 
    )

system_prompt = ("""
    
    - Answer the user's question based on the given context.
    - If the context does not contain the information, do not make up an answer. Simply respond with "I don't know"  
  
    <context>
    {context}
    </context>
    """
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatmodel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
### Here are some ideas for questions you can ask about the document titled "Electrical Cable Routing Design Standard"###

#which extra length should be added to the cables for procurement?
# When are TEL cable routed? 
# -------------------------
#What is the primary purpose of the "Electrical Cable Routing Design Standard" document?
#What software tools are specified in the document for cable routing design?
#What are the key inputs required to perform the cable routing design as per the document?
#Can you explain the difference between manual and auto cable routing methods outlined in the standard?
#What are the responsibilities of engineers when using AVEVA cable design software?

In [8]:
# Query one question 
question = "which extra length should be added to the cables"
input = {'input' : question}
result = rag_chain.invoke(input)

In [11]:
result

{'input': 'which extra length should be added to the cables',
 'context': [Document(metadata={'id': 'page_7', '@search.score': 0.03306011110544205, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, page_content='--- Page7 ---\nPROPRIETARY DOCUMENT - OFFICIAL COPY - ES44331 - PECEELPF447001A1 - PDF Generated on 29-Apr-2022\n11:51AM\nELECTRICAL\nCABLE ROUTING DESIGN\nSTANDARD\nES44331 PECEELPF447001 A 1 PAGE 7 of 14\n4.4 DOCUMENT(S) MANAGEMENT\n4.4.1 Routing Procedure\n4.4.1.1 Required Inputs\nThe following inputs are required to perform the Cable Routing Design:\n3D Model (See paragraph 4.4.1.3).\nDrawings & Documents:\n\uf0b7 Equipment Lists\n\uf0b7 Cable Schedules\n\uf0b7 Key One Line Diagram.\n4.4.1.2 Design Settings\nThe design setting and attributes in Cabling Application Default must be filled up prior to\nstart the cableway and cable design; a standard cable catalogue used by all execution\ncentres shall be selected. This allows the user to se

In [12]:
result['answer']

'For minor equipment, an extra length of 2 meters should be added (e.g., for light fixtures, junction boxes, sockets, and distribution boards). For mechanical workshops/packages/skids/davits, an extra length of 5 meters should be added.'